In [3]:
import numpy as np  
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img  
from keras.models import Sequential  
from keras.layers import Dropout, Flatten, Dense  
from keras import applications  
from keras.utils.np_utils import to_categorical  
import matplotlib.pyplot as plt  
import math  
import cv2  

In [4]:
# dimensions of our images.  
img_width, img_height = 150, 150  

In [5]:
top_model_weights_path = 'bottleneck_fc_model.h5'  
train_data_dir = 'data/train'  
validation_data_dir = 'data/test'  


In [6]:
# number of epochs to train top model  
epochs = 50  
# batch size used by flow_from_directory and predict_generator  
batch_size = 16 

In [7]:
datagen = ImageDataGenerator(rescale=1. / 255)  
   
generator = datagen.flow_from_directory(  
    train_data_dir,  
    target_size=(img_width, img_height),  
    batch_size=batch_size,  
    class_mode=None,  
    shuffle=False)  

Found 2000 images belonging to 2 classes.


In [8]:
nb_train_samples = len(generator.filenames)  # no. of images in train set
num_classes = len(generator.class_indices)   # no. of classes

In [9]:
predict_size_train = int(math.ceil(nb_train_samples / batch_size)) 

In [11]:
model = applications.VGG16(include_top=False, weights='imagenet')

In [13]:
bottleneck_features_train = model.predict(  
     generator, predict_size_train) 

In [14]:
# We do the same for the validation data
generator = datagen.flow_from_directory(  
     validation_data_dir,  
     target_size=(img_width, img_height),  
     batch_size=batch_size,  
     class_mode=None,  
     shuffle=False)

Found 800 images belonging to 2 classes.


In [15]:
nb_validation_samples = len(generator.filenames) 

In [16]:
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))

In [18]:
bottleneck_features_validation = model.predict(  
     generator, predict_size_validation)

In [19]:
np.save('bottleneck_features_validation.npy', bottleneck_features_validation)  

In [20]:
# now we're ready to train our top model
datagen_top = ImageDataGenerator(rescale=1./255)  
generator_top = datagen_top.flow_from_directory(  
        train_data_dir,  
        target_size=(img_width, img_height),  
        batch_size=batch_size,  
        class_mode='categorical',  
        shuffle=False)

Found 2000 images belonging to 2 classes.


In [21]:
nb_train_samples = len(generator_top.filenames)  
num_classes = len(generator_top.class_indices) 

In [22]:
# load the bottleneck features saved earlier  
train_data = np.load('bottleneck_features_train.npy')

In [23]:
# get the class lebels for the training data, in the original order  
train_labels = generator_top.classes

In [24]:
# convert the training labels to categorical vectors  
train_labels = to_categorical(train_labels, num_classes=num_classes)  

In [25]:
#We do the same for validation features
generator_top = datagen_top.flow_from_directory(  
        validation_data_dir,  
        target_size=(img_width, img_height),  
        batch_size=batch_size,  
        class_mode=None,  
        shuffle=False)

Found 800 images belonging to 2 classes.


In [26]:
nb_validation_samples = len(generator_top.filenames) 

In [27]:
validation_data = np.load('bottleneck_features_validation.npy')

In [28]:
validation_labels = generator_top.classes  
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

In [30]:
#  the top model - using the bottleneck features as input, with our classes as the classifier output.
model = Sequential()  
model.add(Flatten(input_shape=train_data.shape[1:]))  
model.add(Dense(256, activation='relu'))  
model.add(Dropout(0.5))  
model.add(Dense(num_classes, activation='sigmoid'))


In [32]:
model.compile(optimizer='adam',  
            loss='categorical_crossentropy', metrics=['accuracy']) 

In [33]:
history = model.fit(train_data, train_labels,  
        epochs=epochs,  
        batch_size=batch_size,  
        validation_data=(validation_data, validation_labels))
# Great !! loss: 0.0258 - accuracy: 0.9870 on train and look at that convergence speed

Epoch 1/50
125/125 [==============================] - 1s 11ms/step - loss: 0.5726 - accuracy: 0.7515 - val_loss: 0.3520 - val_accuracy: 0.8662
Epoch 2/50
125/125 [==============================] - 1s 10ms/step - loss: 0.3876 - accuracy: 0.8500 - val_loss: 0.2898 - val_accuracy: 0.8963
Epoch 3/50
125/125 [==============================] - 1s 10ms/step - loss: 0.2605 - accuracy: 0.8845 - val_loss: 0.2401 - val_accuracy: 0.9075
Epoch 4/50
125/125 [==============================] - 1s 10ms/step - loss: 0.1937 - accuracy: 0.9235 - val_loss: 0.2412 - val_accuracy: 0.9125
Epoch 5/50
125/125 [==============================] - 1s 10ms/step - loss: 0.1624 - accuracy: 0.9310 - val_loss: 0.2393 - val_accuracy: 0.8988
Epoch 6/50
125/125 [==============================] - 1s 10ms/step - loss: 0.1359 - accuracy: 0.9410 - val_loss: 0.2352 - val_accuracy: 0.9150
Epoch 7/50
125/125 [==============================] - 1s 11ms/step - loss: 0.1375 - accuracy: 0.9435 - val_loss: 0.2398 - val_accuracy: 0.9075

In [34]:
(eval_loss, eval_accuracy) = model.evaluate(  
     validation_data, validation_labels, batch_size=batch_size, verbose=1)

50/50 [==============================] - 0s 3ms/step - loss: 0.4394 - accuracy: 0.8963


In [35]:
print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))

[INFO] accuracy: 89.63%


In [36]:
print("[INFO] Loss: {}".format(eval_loss))

[INFO] Loss: 0.439431369304657
